In [1]:
## importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
loan = pd.read_csv('loan.csv')

/Applications/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [4]:
loan.shape

(39717, 111)

In [ ]:
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


## Data Cleaning

### 1. Checking for Missing Values

The dataset has 111 columns. We will ignore the columns that have all values as null as they will not be helpful for our analysis

In [ ]:
# columns that have all values as null
for x in loan.columns:
    if loan[x].isnull().sum()>=39717:
        print(x)

In [ ]:
columns_notnull = [x for x in loan.columns if loan[x].isnull().sum()<39717]
len(columns_notnull) # columns that do not have all values as null

Checking if there are columns that have only one value across all the rows.
These rows are also not useful as they dont indicate any significant information

In [ ]:
[x for x in columns_notnull if len(loan[x].value_counts())<=1]

In [ ]:
#Checking the unique values for each column that has only one unique value
for x in columns_notnull:
    if len(loan[x].value_counts())<=1:
        print('Unique Values in {} are {}'.format(x,loan[x].value_counts().index[0]))

We can safely ignore these columns too for our analysis

In [ ]:
# removing the columns that have same value
columns_notnull = [x for x in columns_notnull if len(loan[x].value_counts())>1]
len(columns_notnull)

In [ ]:
for x in columns_notnull:
    if loan[x].isnull().sum()>0:
        print(x, loan[x].isnull().sum())

Now that we have 48 columns, There are 7 columns that have null values. We will review the columns that have null values one by one

In [ ]:
for x in columns_notnull:
    if loan[x].isnull().sum()>0:
        print(x, loan[x].isnull().sum())

#### emp_title : "The job title supplied by the Borrower when applying for the loan."

1. emp_title has 2458 null values
2. Also in the data dictionary it is mentioned that "Employer Title replaces Employer Name for all loans listed after 9/23/2013" 

But there is no column indicating employer name, hence we will add all null values under a category 'Other'

In [ ]:
#Checking if 'Other' value is already present
loan[loan.emp_title.str.lower()=='other'].emp_title

In [ ]:
#filling all null values as 'Other'
loan.emp_title.fillna('Other', inplace=True)
loan.emp_title.isnull().sum() # checking if all null values are filled

#### desc : "loan description provided by the borrower"

1. desc column has 12939 null values. 
2. We can fill all the null values with 'Not Provided'

In [ ]:
loan.desc[3]

In [ ]:
loan.desc.fillna('Not Provided', inplace=True)
loan.desc.isnull().sum() # checking if all null values are filled

#### title : "The loan title provided by the borrower"

1. title has 10 null values
2. We can fill the null values as 'Not Provided'

In [ ]:
loan.title[0:10]

In [ ]:
loan.title.fillna('Not Provided', inplace=True)
loan.title.isnull().sum()

#### mths_since_last_delinq : "The number of months since the borrower's last delinquency."

1. mths_since_last_delinq has 25682 null values
2. As per the description of the field, this column indicates the number of months if borrower is delinquent

In [ ]:
print ('Percentage of null values present = {}%. There are {} null values in this field out of {} values'
       .format(round(100* loan[loan.mths_since_last_delinq.isnull()].shape[0]/loan.shape[0],2), 
               loan[loan.mths_since_last_delinq.isnull()].shape[0], 
               loan.shape[0])
      )

Hence removing this column from our analysis

In [ ]:
columns_notnull.remove('mths_since_last_delinq')

#### mths_since_last_record : "The number of months since the last public record."

1. mths_since_last_record has 36931 null values

Lets look at the percentage of null values

In [ ]:
print ('Percentage of null values present = {}%. There are {} null values in this field out of {} values'
       .format(round(100* loan[loan.mths_since_last_record.isnull()].shape[0]/loan.shape[0],2), 
               loan[loan.mths_since_last_record.isnull()].shape[0], 
               loan.shape[0])
      )

Hence removing this column from our analysis

In [ ]:
columns_notnull.remove('mths_since_last_record')

#### revol_util : "Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit."

1. There are 50 null values in revol_util column
2. This column indicates the revolving line utilization rate.
3. We can replace all the null values to 0% (min value) so that we can utilize the not-null values during our analysis

In [ ]:
print ('Percentage of null values present = {}%. There are {} null values in this field out of {} values'
       .format(round(100* loan[loan.revol_util.isnull()].shape[0]/loan.shape[0],2), 
               loan[loan.revol_util.isnull()].shape[0], 
               loan.shape[0])
      )

In [ ]:
loan.revol_util.fillna('0',inplace=True)
loan.revol_util.isnull().sum() # checking if the null values are replaced

#### last_pymnt_d : "Last month payment was received"

1. This column has 71 null values

In [ ]:
loan[loan.last_pymnt_d.isnull()>0].last_pymnt_amnt.value_counts()

In [ ]:
loan.last_pymnt_d.value_counts()

2. The blank values in this column indicates that last month payment was not received as the last_pymnt_amnt is null.
3. We decide to keep the null values as is for our further analysis

#### next_pymnt_d : "Next scheduled payment date"

1. next_pymnt_d has 38577 null values
2. Next payment date will be scheduled for loans that are not yet paid fully. So let us look at the loan status to compare this field

In [ ]:
loan[loan.next_pymnt_d.isnull()>0].loan_status.value_counts()

all null values in next_pymnt_d fall under 'Fully Paid' or 'Charged Off' under loan_status column. we have to keep this column to analyze next_pymnt_d on other loan statuses

#### last_credit_pull_d : "The most recent month LC pulled credit for this loan"

1. last_credit_pull_d has 2 null values
2. Since the null values are only 2, We will keep this column for our analysis

In [ ]:
loan.last_credit_pull_d.describe()

#### pub_rec_bankruptcies : "Number of public record bankruptcies"

1. There are 697 null values in this field

In [ ]:
loan.pub_rec_bankruptcies.describe()

Since the minimum is 0 for this column and maximum is 2, Let us fill the null values with 0 (no records)

In [ ]:
loan.pub_rec_bankruptcies.fillna(0,inplace=True)

Now the data is cleaned for missing values and all the necessary columns names are captured in the list columns_notnull

In [ ]:
print('We have the below {} columns after cleaning the dataset for missing values \n \n {}'
      .format(len(columns_notnull),columns_notnull))

### 2. Delete columns that are unnecessary

#### id: A unique LC assigned ID for the loan listing.

In [ ]:
print("Number of unique values in 'id' column = {}".format(len(loan.id.unique())))

In the loan dataset, the column 'id' is the unique identifier for the LC and all the values are unique. 
Let us remove the column for our analysis

In [ ]:
columns_notnull.remove('id')

#### member_id: A unique LC assigned Id for the borrower member.

In [ ]:
print("Number of unique values in 'member_id' column = {}".format(len(loan.member_id.unique())))

Similar to the 'id' column, member_id also has all unique values. So let us remove the column for our analysis

In [ ]:
columns_notnull.remove('member_id')

In [ ]:
print("We have the below {} columns after 'deleting the identified columns' \n \n {}".format(len(columns_notnull),columns_notnull))

In [ ]:
# loan dataset is sliced to have only the above columns
loan = loan[columns_notnull]

In [ ]:
loan.shape

### 3. Convert incorrect data types:

#### 3.1 There are date columns in the dataset that needs to be corrected to proper datetime format

['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']

#### issue_d

In [ ]:
loan.issue_d.value_counts()[0:10]

The issue_d has only month and year and not the date of the month.

In [ ]:
loan.last_pymnt_d.value_counts()[0:10]

In [ ]:
loan.next_pymnt_d.value_counts()[0:10]

In [ ]:
loan.last_credit_pull_d.value_counts()[0:10]

Similar to issue_d, all the above dates contain only month and Year. 

In [ ]:
loan['issue_d'] = pd.to_datetime(loan.issue_d,format='%b-%y')
loan['last_pymnt_d'] = pd.to_datetime(loan.last_pymnt_d,format='%b-%y')
loan['next_pymnt_d'] = pd.to_datetime(loan.next_pymnt_d,format='%b-%y')
loan['last_credit_pull_d'] = pd.to_datetime(loan.last_credit_pull_d,format='%b-%y')

In [ ]:
#converted dates to correct format
loan[['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']].head()

#### 3.2 Check and convert datatypes object to appropriate format

In [ ]:
loan.select_dtypes(include=['object']).columns

##### term has 'months'

In [ ]:
loan.term[0]

In [ ]:
# Removing 'months' from term
loan['term'] = loan.term.apply(lambda x: int(x.lstrip(' ').rstrip(' months')))
loan.term.value_counts()

##### int_rate has % and also needs to be converted to fraction

In [ ]:
loan.int_rate[0]

In [ ]:
#removing % and converting to fraction, rounded to two decimal digits
loan['int_rate'] = loan.int_rate.apply(lambda x: round(float(x.rstrip('%'))/100,2))
loan.int_rate.value_counts()

##### grade values are categorical

In [ ]:
loan.grade.value_counts()

##### sub_grade is also categorical

In [ ]:
loan.sub_grade.value_counts()

##### emp_title is also categorical

In [ ]:
loan.emp_title.value_counts()

##### emp_length has + sign and years prefixed

In [ ]:
loan.emp_length[0]

In [ ]:
loan.emp_length.value_counts()

In [ ]:
loan['emp_length'] = loan.emp_length.apply(lambda x: int(x.replace('+','')
                                                         .replace(' years','')
                                                         .replace('< ','')
                                                         .replace(' year','')
                                                         .replace('n/a','0'))
                                        )
loan.emp_length.value_counts()

##### home_ownership is categorical

In [ ]:
loan.home_ownership.value_counts()

##### verification_status  is categorical

In [ ]:
loan.verification_status.value_counts()

##### loan_status is categorical

In [ ]:
loan.loan_status.value_counts()

##### url is categorical

In [ ]:
loan.url.unique()

In [ ]:
# Extracting information other than "https://lendingclub.com/browse/loanDetail.action?loan_id="
loan['url']= loan.url.apply(lambda x: int(x.replace('https://lendingclub.com/browse/loanDetail.action?loan_id=','')))
loan.url.unique()

After removing the url common path from the 'url' column, the column only contains unique identifiers. 
Hence we can remove this column from our analysis as we have also excluded 'id' and 'member_id' columns

In [ ]:
loan = loan.drop('url',axis=1)

##### desc is categorical

In [ ]:
loan.desc.unique()

We will convert 'desc' column to two values - Provided, Not Provided. As the analysis can be done based on whether a description was provided during the loan application or not

In [ ]:
# def func(x):
#     if x=='Not Provided':
#         return x
#     else return 'Provided'
loan['desc'] = loan.desc.apply(lambda x: x if (x=='Not Provided') else 'Provided')
loan.desc.value_counts()

##### purpose is categorical

In [ ]:
loan.purpose.value_counts()

##### title is categorical

In [ ]:
loan.title.value_counts()

##### zip_code is categorical

In [ ]:
loan.zip_code.unique()

In [ ]:
#changing zip_code column to zip_code_first3 and removing xx 
loan['zip_code_first3']= loan.zip_code.apply(lambda x: x.rstrip('xx'))
loan.zip_code_first3.unique()

##### addre_state is categorical

In [ ]:
loan.addr_state.unique()

##### earlier_cr_line is categorical

In [ ]:
loan.earliest_cr_line.unique()

earliest_cr_line needs to be converted to datetime format

In [ ]:
loan['earliest_cr_line'] = pd.to_datetime(loan.earliest_cr_line,format='%b-%y')
loan.earliest_cr_line.dtype

##### revol_util is categorical

In [ ]:
loan.revol_util.unique()

In [ ]:
# percentage % needs to be removed and converted to fraction rounded upto 2 decimal places
loan['revol_util'] = loan.revol_util.apply(lambda x: float(x.replace('%',''))/100)

Now lets check if we have successfully treated all the object dtypes in the dataset

In [ ]:
loan.select_dtypes(include=['object']).columns

Also to check if there are any other dtypes needs to be treated other than object

In [ ]:
loan.select_dtypes(exclude=['object','datetime64','float','int']).columns

### 4. Validate internal rules

#### 4.1 Date validation: issue_d is older than last_pymnt_d

In [ ]:
loan[loan.issue_d > loan.last_pymnt_d]

No values found where issue_d is earlier than last_pymnt_d

#### 4.2 Date validation: issue_d is older than next_pymnt_d

In [ ]:
loan[loan.issue_d > loan.last_pymnt_d]

No values found where issue_d is earlier than next_pymnt_d

#### 4.3 Date validation: last_pymnt_d is earlier than next_payment_d

In [ ]:
loan[loan.last_pymnt_d > loan.next_pymnt_d]

No values found where last_pymnt_d is earlier than next_payment_d

#### loan dataset is now treated/cleaned for further analysis

In [ ]:
loan.info()

## Analysis